- 贾承斌
    - 接口和定时聚类程序都试下数据

# 基本设置

In [1]:
import pandas as pd
import sys

import langid # 语种检测
langid.classify("I do not speak english")[0]

'en'

In [68]:
import os
pwd = os.getcwd()
print(pwd)
sys.path.append(pwd)

D:\XH\Python_Project\notebook\tmp_proj\clustering_jiachengbing


In [22]:
class ClusterMessageObj(object):
    """
    聚类实体类
    """

    def __init__(self, messageId=0, messageTitle='', messagePublishtime='', messageContent='', site_name=''):
        self.messageId = messageId
        self.messageTitle = messageTitle
        self.messagePublishtime = messagePublishtime
        self.messageContent = messageContent
        self.site_name = site_name

In [29]:
def parse_corpus_records(records, language_type=None):
    data = []
    for row in records:
        id = row['id']

        title = row['title']
        title = title.encode('utf-8')

        content = row['content']
        content = content.encode('utf-8')

        publish_time = row['publishtime'].encode('utf-8') # .strftime("%Y-%m-%d %H:%M:%S")
        site_name = row['site_name']
        site_name = site_name.encode('utf-8')

        cluster_obj = ClusterMessageObj(id, title, publish_time, content, site_name)

        # 进行语言过滤
        if language_type is not None:
            if BaseDataView.is_valid_language(language_type, content):
                data.append(cluster_obj)
        else:
            data.append(cluster_obj)
    return data

In [2]:
sys.getdefaultencoding()

'utf-8'

In [49]:
from toolkits.nlp.langconv import *

def Traditional2Simplified(sentence):
    '''
    将sentence中的繁体字转为简体字
    :param sentence: 待转换的句子
    :return: 将句子中繁体字转换为简体字之后的句子
    '''
    sentence = Converter('zh-hans').convert(sentence)
    return sentence

# load data

In [16]:
data = pd.read_excel(u'外媒_20190426070934.xlsx')
print(data.shape)
print(data.columns)
data.iloc[:1, :]

(9641, 15)
Index(['id', '来源id', '来源', '标题', '发布时间', '链接', '作者', '点击', '点赞', '回复', '转发',
       '查询关键词', '正文文件名称', '正文', '类型'],
      dtype='object')


,id,来源id,来源,标题,发布时间,链接,作者,点击,点赞,回复,转发,查询关键词,正文文件名称,正文,类型
0,1,299507,华尔街日报中文网,隔夜市场回顾,2019-04-24 09:45:00.0,https://cn.wsj.com/articles/隔夜市场回顾-11556062811,NaN,0,0,0,0,cn.wsj.com,NaN,美股收盘上涨，标普500和纳指双双收于纪录高点；美元升至今年高点，因房地产数据强劲；美国国债...,外媒


## 检测语种

In [17]:
data['language'] = data['标题'].astype('str').apply(lambda x: langid.classify(x)[0])

In [18]:
print(data.shape)
data.iloc[:1, :]

(9641, 16)


,id,来源id,来源,标题,发布时间,链接,作者,点击,点赞,回复,转发,查询关键词,正文文件名称,正文,类型,language
0,1,299507,华尔街日报中文网,隔夜市场回顾,2019-04-24 09:45:00.0,https://cn.wsj.com/articles/隔夜市场回顾-11556062811,NaN,0,0,0,0,cn.wsj.com,NaN,美股收盘上涨，标普500和纳指双双收于纪录高点；美元升至今年高点，因房地产数据强劲；美国国债...,外媒,zh


In [21]:
# 繁体
data[data['id'] == 7914]

,id,来源id,来源,标题,发布时间,链接,作者,点击,点赞,回复,转发,查询关键词,正文文件名称,正文,类型,language
7913,7914,299138,联合新闻网,影／網路叫車平台代僱駕自救會 赴交通部抗議,2019-04-25 15:21:30.0,https://udn.com/news/story/7266/3776682,NaN,0,0,0,0,udn.com,NaN,網路叫車平台代僱駕自救會今天到交通前陳情，高喊口號呼籲重視網路叫車平台代僱駕自救會司機的權益...,外媒,zh


In [19]:
data['language'].value_counts()

en    3941
zh    3346
ja    1022
pt     465
ar     299
tl     165
id      48
de      43
es      40
fr      39
it      35
nl      24
da      20
gl      19
xh      12
no      10
sv      10
pl       9
rw       9
af       9
sw       7
mt       6
ms       6
la       6
et       6
ca       5
jv       5
lt       4
ro       4
tr       3
eu       3
hu       3
se       2
ur       2
fi       2
mg       2
nn       2
oc       1
ku       1
vo       1
sl       1
sq       1
fa       1
eo       1
nb       1
Name: language, dtype: int64

## 英文数据

In [94]:
data_en = data[data['language'] == 'en'][['id', '标题', '正文', '发布时间', '来源']]
data_en.columns = ['id', 'title', 'content', 'publishtime', 'site_name']
print(data_en.shape)
data_en.iloc[:1, :]

(3941, 5)


,id,title,content,publishtime,site_name
42,43,China Is Not Interested in War with America,Editor’s note: This essay was a runner up in...,2019-04-25 01:19:17.0,国家利益杂志


In [95]:
file_path =  "mlUtil_MOE/corpus/corpus_en_20190426070934.json"
with open(file_path,'w',encoding='utf-8') as json_file:
        json.dump(data_en.to_dict('records'),json_file,ensure_ascii=False)  

In [36]:
# corpus_en = parse_corpus_records(data_en.to_dict('records'), language_type=None)
# dir(corpus_en[0])
# corpus_en[0].messageTitle

b'China Is Not Interested in War with America'

In [89]:
# from sklearn.externals import joblib
# import pickle
# file_path =  "mlUtil_MOE/corpus/corpus_en_20190426070934.pkl"
# pickle.dump(corpus_en, file_path)
# pipeline_train = joblib.load( "model/previous_model/cbrc_pipeline_20181108.pkl.z")

In [88]:
# py2 与 py3 兼容
# # 在python3中将数据格式转换成python2可读的数据格式
# with open(file_path, 'rb') as f:
#     w = pickle.load(f)
# pickle.dump(w, open(file_path,"wb"), protocol=2)

## 中文数据

In [96]:
data_zh = data[data['language'] == 'zh'][['id', '标题', '正文', '发布时间', '来源']]
data_zh.columns = ['id', 'title_raw', 'content_raw', 'publishtime', 'site_name']
# 繁体转简体
data_zh['title'] = data_zh['title_raw'].apply(lambda x: Traditional2Simplified(x))
data_zh['content'] = data_zh['content_raw'].apply(lambda x: Traditional2Simplified(x))
print(data_zh.shape)
data_zh.iloc[:1, :]

(3346, 7)


,id,title_raw,content_raw,publishtime,site_name,title,content
0,1,隔夜市场回顾,美股收盘上涨，标普500和纳指双双收于纪录高点；美元升至今年高点，因房地产数据强劲；美国国债...,2019-04-24 09:45:00.0,华尔街日报中文网,隔夜市场回顾,美股收盘上涨，标普500和纳指双双收于纪录高点；美元升至今年高点，因房地产数据强劲；美国国债...


In [97]:
data_zh.loc[7914, :]

id                                                          7915
title_raw                                  長榮航空勞資協商破局 林佳龍：仍有可談空間
content_raw    長榮航空勞資調解沒共識，桃園市空服員職業工會宣布5月13日啟動罷工投票，圖為長榮空服員。 記...
publishtime                                2019-04-25 15:25:27.0
site_name                                                  联合新闻网
title                                      长荣航空劳资协商破局 林佳龙：仍有可谈空间
content        长荣航空劳资调解没共识，桃园市空服员职业工会宣布5月13日启动罢工投票，图为长荣空服员。 记...
Name: 7914, dtype: object

In [101]:
file_path =  "mlUtil_MOE/corpus/corpus_zh_20190426070934.json"
data_zh.to_excel(file_path.replace('.json', '.xlsx'), index = False)
with open(file_path,'w',encoding='utf-8') as json_file:
        json.dump(data_zh.to_dict('records'),json_file,ensure_ascii=False)  

In [57]:
# corpus_zh = parse_corpus_records(data_zh.to_dict('records'), language_type=None)
# dir(corpus_zh[0])
# corpus_zh[0].messageTitle.decode('utf-8')

'隔夜市场回顾'

In [82]:
# from sklearn.externals import joblib
# joblib.dump(corpus_zh, "mlUtil_MOE/corpus/corpus_en_20190426070934.pkl.z")

['mlUtil_MOE/corpus/corpus_en_20190426070934.pkl.z']

# 接口结果

# 定时结果

## 中文聚类 DBSCAN

## 英文聚类

In [93]:
from sklearn.externals import joblib
file_path = 'mlUtil_MOE/result/result_en_20190426070934.json'
result_en = joblib.load(file_path)

ModuleNotFoundError: No module named 'entity.cluster_result'